#Common Setting

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/security_mft

/content/drive/MyDrive/GitHub/security_mft


In [3]:
%pwd
%ls

code/  dfxml_files/  MFT_files/  README.md  resident_files/


In [4]:
import pandas as pd
import numpy as np
import os
import glob
import sys
import hashlib
import re
from datetime import datetime
import time

import xml.etree.ElementTree as ET


In [8]:
start_time = datetime.now()
# mode = "test" #run or test
mode = "run" #run or test

env = 1 # 1- google colab,  else - local
if(env == 1):
  pathToDxml = "/content/drive/MyDrive/security_mft/dfxml_files"
else:
  pathToDxml = "."

output_path = '/content/drive/MyDrive/security_mft/output/'

In [ ]:
# def getTag(tree):
#   tree = ET.parse('first_second.dfxml')
#   tagList = []
#   for elem in tree.iter():
#     tagList.append(re.sub(r"\{[^()]*\}", "", elem.tag))
#   #tagList = list(set(tagList))
#   return tagList

In [ ]:
#test 1
s = '{http://www.forensicswiki.org/wiki/Category:Digital_Forensics_XML}gid'
s = re.sub(r'{.+}', '', s)
print(f"after: {s}")

after: gid


#Method 1 ElementTree

In [ ]:
def parseDxml(pathToFile):
  start_time = datetime.now()

  tree = ET.parse(pathToFile)
  root = tree.getroot()
  df = showInfo(root)
  csv_filename = f"{pathToDxml}/parse_result_{os.path.basename(pathToFile)[:-6]}.csv"
  df.to_csv (csv_filename, index = False, header=True)

  end_time = datetime.now()
  print(f"Duration of parsing data for {os.path.basename(pathToFile)[:-6]}: {end_time - start_time}")



In [ ]:
import re

# #test 1
# s = '{http://www.forensicswiki.org/wiki/Category:Digital_Forensics_XML}gid'
# s = re.sub(r'{.+}', '', s)
# print(f"after: {s}")

# #test 2
# s = '<@ """@$ FSDF >something something <more noise>'
# s = re.sub(r'<.+>', '',s)
# # s = re.sub('<[^>]+>', '', s)
# print(f"after: {s}")

def rename(s):
  s = re.sub(r'{.+}', '', s)
  # print(s)
  return s

In [ ]:
def updt(total, progress):
    """
    Displays or updates a console progress bar.

    Original source: https://stackoverflow.com/a/15860757/1391441
    """
    barLength, status = 20, ""
    progress = float(progress) / float(total)
    if progress >= 1.:
        progress, status = 1, "\r\n"
    block = int(round(barLength * progress))
    text = "\r[{}] {:.0f}% {}".format(
        "#" * block + "-" * (barLength - block), round(progress * 100, 0),
        status)
    sys.stdout.write(text)
    sys.stdout.flush()

'''
runs = 300
for run_num in range(runs):
    time.sleep(.1)
    updt(runs, run_num + 1)
'''

'\nruns = 300\nfor run_num in range(runs):\n    time.sleep(.1)\n    updt(runs, run_num + 1)\n'

In [ ]:
setDuplicate = set()
# parentNodeInfo = None
def dictAssign(tempDict, key, item):
  # print(f"{tempDict}")
  # print(f"{key}")
  # print(f"{item}")
  global setDuplicate
  # global parentNodeInfo

  #if duplicate
  if(key in tempDict.keys()):
    if(tempDict[key]):
      setDuplicate.add(key)
      # print(parentNodeInfo)
      if(type(tempDict[key]) is list):
        tempDict[key].append(item)
      else:
        firstValue = tempDict[key]
        tempDict[key] = [firstValue]
        tempDict[key].append(item)
    else:
      #first time, not duplicate
      tempDict[key] = item
  else:
    tempDict[key] = item

  return tempDict

In [ ]:




def showInfo(root):
  
  test = 0
  listColumns = ['fileobject_id']
  df = pd.DataFrame(columns=listColumns)
  fileobjects = root.findall('.//{http://www.forensicswiki.org/wiki/Category:Digital_Forensics_XML}fileobject')
  for fileobject_id, fileobject in enumerate(fileobjects):
    tempDict = {}
    # tempDict['fileobject_id'] = fileobject_id
    tempDict = dictAssign(tempDict,"fileobject_id",fileobject_id)
    for node in fileobject.iter():
      # global parentNodeInfo
      # if(node.getparent()):
      #   parentNodeInfo = node.getparent().tag
      # if(mode == 'test'):
      #   pass
        # if(test < 1):
        #   print(f"id: {fileobject_id}, tag: {rename(node.tag)}")
        #   print(f"id: {fileobject_id}, text: {node.text}")
        #   print(f"id: {fileobject_id}, attributes: {node.attrib}")
      # print(rename(node.tag))
      # tempDict[rename(node.tag)] = node.text
      tempDict = dictAssign(tempDict,rename(node.tag),node.text)

      # tempDict['attrib'] = node.attrib
      if(node.attrib): #if the attrib exists
        for key, item in node.attrib.items():
          # print(f"attrib: key: {key}, item: {item}")
          tempDict = dictAssign(tempDict,rename(key),item)
          # tempDict[rename(key)] = item
    df = df.append(tempDict, 
              ignore_index=True)


    test+=1

    updt(len(fileobjects), fileobject_id + 1)
    if(mode == 'test'):
      if(test>2):
        print(f"fileobject 0 ~ {fileobject_id} is done. Then break...")
        break

  return df
      
      



In [ ]:
test = 0
for eachFile in glob.glob(f"{pathToDxml}/*.dfxml"):
  print(f"parsing {eachFile} ...")
  # if (os.path.exists(f"{pathToDxml}/parse_result_{os.path.basename(eachFile)[:-6]}.csv")):
  #   print(f"{pathToDxml}/parse_result_{os.path.basename(eachFile)[:-6]}.csv exists, move to next dfxml file.")
  #   continue
  
  # print(f"size (bytes): {os.path.getsize(eachFile)}")
  # print(f"basename (no extention): {os.path.basename(eachFile)[:-6]}")
  parseDxml(eachFile)
  test+=1
  if(mode == "test"):
    if(test==0):
      break
print(f"the following columns has duplicated values: {setDuplicate}")
print(f"value of the len(setDuplicate): {len(setDuplicate)}")

parsing /content/drive/MyDrive/security_mft/dfxml_files/fourth_fifth.dfxml ...
[--------------------] 1% fileobject 0 ~ 2 is done. Then break...
Duration of parsing data for fourth_fifth: 0:00:00.055884
parsing /content/drive/MyDrive/security_mft/dfxml_files/sixth_seventh.dfxml ...
[--------------------] 0% fileobject 0 ~ 2 is done. Then break...
Duration of parsing data for sixth_seventh: 0:00:00.579759
parsing /content/drive/MyDrive/security_mft/dfxml_files/second_third.dfxml ...
[--------------------] 0% fileobject 0 ~ 2 is done. Then break...
Duration of parsing data for second_third: 0:00:00.237946
parsing /content/drive/MyDrive/security_mft/dfxml_files/first_second.dfxml ...
[--------------------] 0% fileobject 0 ~ 2 is done. Then break...
Duration of parsing data for first_second: 0:00:00.477721
parsing /content/drive/MyDrive/security_mft/dfxml_files/fifth_sixth.dfxml ...
[--------------------] 0% fileobject 0 ~ 2 is done. Then break...
Duration of parsing data for fifth_sixth: 

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
if(mode == "test"):
  filepath = glob.glob(f"{pathToDxml}/parse_result_fourth_fifth.csv")[0]#"/content/drive/MyDrive/security_mft/dfxml_files/parse_result_fourth_fifth.csv"
else:
  filepath = glob.glob(f"{pathToDxml}/*.csv")[0]#"/content/drive/MyDrive/security_mft/dfxml_files/parse_result_fourth_fifth.csv"
print(filepath)
df = pd.read_csv(filepath)
print(f"How many columns: {len(df.columns)}")
print(f"column names: {df.columns}")
print(df.head(50))

/content/drive/MyDrive/security_mft/dfxml_files/parse_result_fourth_fifth.csv
How many columns: 34
column names: Index(['fileobject_id', 'alloc', 'atime', 'byte_run', 'byte_runs', 'crtime',
       'ctime', 'file_offset', 'filename', 'fileobject', 'filesize',
       'fs_offset', 'gid', 'hashdigest', 'id', 'img_offset', 'inode', 'len',
       'meta_type', 'mode', 'mtime', 'name_type', 'new_file', 'nlink',
       'parent_object', 'partition', 'seq', 'type', 'uid', 'used',
       'changed_file', 'changed_property', 'deleted_file',
       'original_fileobject'],
      dtype='object')
   fileobject_id       alloc  \
0              0           0   
1              1  ['0', '1']   
2              2  ['0', '1']   

                                              atime  byte_run  byte_runs  \
0                              2021-06-06T20:07:19Z       NaN        NaN   
1  ['2021-06-06T20:11:52Z', '2021-06-06T20:11:52Z']       NaN        NaN   
2  ['2021-06-06T20:11:52Z', '2021-06-06T20:11:52Z']      

In [ ]:
end_time_total = datetime.now()
print('Duration of finishing parsing all dfxml files: {}'.format(end_time_total - start_time))

Duration of finishing parsing all dfxml files: 0:00:02.692491




```
# This is formatted as code
```

# Method 2: lxml (can access parent node)

In [6]:
from lxml import etree


In [ ]:
def parseDxml_lxml(pathToFile):
  start_time = datetime.now()

  tree = etree.parse(pathToFile)
  root = tree.getroot()
  df, byteRunDf = showInfo_lxml(root)

  csv_filename = f"{pathToDxml}/parse_result_{os.path.basename(pathToFile)[:-6]}.csv"
  csv_filename_byte_run = f"{pathToDxml}/parse_result_{os.path.basename(pathToFile)[:-6]}_byte_runs.csv"
  df.to_csv (csv_filename, index = False, header=True)
  byteRunDf.to_csv (csv_filename_byte_run, index = False, header=True)

  end_time = datetime.now()
  print(f"Duration of parsing data for {os.path.basename(pathToFile)[:-6]}: {end_time - start_time}")

In [ ]:
setDuplicate = set()
parentNodeInfo = dict()
def dictAssign(tempDict, key, item):
  # print(f"{tempDict}")
  # print(f"{key}")
  # print(f"{item}")
  global setDuplicate
  global parentNodeInfo
  nodeIter = 1
  #if duplicate
  '''
  if(key != "fileobject_id"):
    if(f"{key}_{parentNodeInfo[key]}" in tempDict.keys()):
      nodeIter = nodeIter + 1
      strSpecify = f"{key}_{parentNodeInfo[key]}_{nodeIter}"
      tempDict[strSpecify] = item
      return tempDict, nodeIter
  '''
  if(key in tempDict):
    while True:
      if(f"{key}_{nodeIter}" in tempDict):
        nodeIter = nodeIter+1
      else:
        break
    # if(tempDict[key]):
    setDuplicate.add(key)
    # print(parentNodeInfo)
    # nodeIter = nodeIter + 1
    strFirst = f"{key}_{nodeIter}"
    strSpecify = f"{key}_{nodeIter}"
    # tempDict[strFirst] = tempDict[key]
    tempDict[strSpecify] = item

    '''
      #store to a list
      if(type(tempDict[key]) is list):
        tempDict[key].append(item)
        strFirst = f"{key}_first"
        strSpecify = f"{key}_{parentNodeInfo}"
        tempDict[strFirst] = tempDict[key][0]
        tempDict[strSpecify] = item
      else:
        firstValue = tempDict[key]
        tempDict[key] = [firstValue]
        tempDict[key].append(item)
    '''
    '''
    else:
      #first time, not duplicate
      tempDict[key] = item
    '''
  else:
    tempDict[key] = item
    nodeIter = 0

  return tempDict, nodeIter

In [ ]:
def showInfo_lxml(root):
  
  test = 0
  listColumns = ['fileobject_id']
  df = pd.DataFrame(columns=listColumns)
  byteRunDf = pd.DataFrame(columns=listColumns)
  fileobjects = root.findall('.//{http://www.forensicswiki.org/wiki/Category:Digital_Forensics_XML}fileobject')
  for fileobject_id, fileobject in enumerate(fileobjects):
    tempDict = {}
    byteRunTempDict = {}
    # tempDict['fileobject_id'] = fileobject_id
    
    tempDict = dictAssign(tempDict,"fileobject_id",fileobject_id)[0]
    
    for node in fileobject.iter():
      if(rename(node.tag) == "byte_run"):
        byteRunTempDict["fileobject_id"] = fileobject_id
        byteRunTempDict[rename(node.tag)] = node.text

        for key, item in node.attrib.items():
          byteRunTempDict[rename(key)] = item
        byteRunDf = byteRunDf.append(byteRunTempDict, ignore_index=True)

      else:
        global parentNodeInfo
        if(node.getparent()):
          # if(key in tempDict.keys())
          parentNodeInfo[rename(node.tag)] = rename(node.getparent().tag)
        else:
          parentNodeInfo[rename(node.tag)] = None

        # if(mode == 'test'):
        #   pass
          # if(test < 1):
          #   print(f"id: {fileobject_id}, tag: {rename(node.tag)}")
          #   print(f"id: {fileobject_id}, text: {node.text}")
          #   print(f"id: {fileobject_id}, attributes: {node.attrib}")
        # print(rename(node.tag))
        # tempDict[rename(node.tag)] = node.text
        tempDict, nodeIter = dictAssign(tempDict,rename(node.tag),node.text)

        # tempDict['attrib'] = node.attrib
        if(node.attrib): #if the attrib exists
          for key, item in node.attrib.items():
            # print(f"attrib: key: {key}, item: {item}")
            # tempDict = dictAssign(tempDict,rename(key),item)
            if(nodeIter != 0):
              tempDict[f'{rename(key)}_{rename(node.tag)}_{nodeIter}'] = item
            else:
              tempDict[f'{rename(key)}_{rename(node.tag)}'] = item
    df = df.append(tempDict, ignore_index=True)
    


    test+=1

    updt(len(fileobjects), fileobject_id + 1)
    if(mode == 'test'):
      if(test>20):
        print(f"fileobject 0 ~ {fileobject_id} is done. Then break...")
        break

  return df, byteRunDf


In [ ]:
test = 0
for eachFile in glob.glob(f"{pathToDxml}/*.dfxml"):
  print(f"parsing {eachFile} ...")
  # if (os.path.exists(f"{pathToDxml}/parse_result_{os.path.basename(eachFile)[:-6]}.csv")):
  #   print(f"{pathToDxml}/parse_result_{os.path.basename(eachFile)[:-6]}.csv exists, move to next dfxml file.")
  #   continue
  
  # print(f"size (bytes): {os.path.getsize(eachFile)}")
  # print(f"basename (no extention): {os.path.basename(eachFile)[:-6]}")
  parseDxml_lxml(eachFile)
  test+=1
  if(mode == "test"):
    if(test==0):
      break
print(f"the following columns has duplicated values: {setDuplicate}")
print(f"value of the len(setDuplicate): {len(setDuplicate)}")

parsing /content/drive/MyDrive/security_mft/dfxml_files/fourth_fifth.dfxml ...
[--------------------] 1% fileobject 0 ~ 2 is done. Then break...
Duration of parsing data for fourth_fifth: 0:00:00.058678
parsing /content/drive/MyDrive/security_mft/dfxml_files/sixth_seventh.dfxml ...
[--------------------] 0% 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.


[--------------------] 0% fileobject 0 ~ 2 is done. Then break...
Duration of parsing data for sixth_seventh: 0:00:00.179903
parsing /content/drive/MyDrive/security_mft/dfxml_files/second_third.dfxml ...
[--------------------] 0% fileobject 0 ~ 2 is done. Then break...
Duration of parsing data for second_third: 0:00:00.099606
parsing /content/drive/MyDrive/security_mft/dfxml_files/first_second.dfxml ...
[--------------------] 0% fileobject 0 ~ 2 is done. Then break...
Duration of parsing data for first_second: 0:00:00.132036
parsing /content/drive/MyDrive/security_mft/dfxml_files/fifth_sixth.dfxml ...
[--------------------] 0% fileobject 0 ~ 2 is done. Then break...
Duration of parsing data for fifth_sixth: 0:00:00.288031
parsing /content/drive/MyDrive/security_mft/dfxml_files/third_fourth.dfxml ...
[--------------------] 0% fileobject 0 ~ 2 is done. Then break...
Duration of parsing data for third_fourth: 0:00:00.117707
the following columns has duplicated values: {'id', 'mode', 'type

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
if(mode == "test"):
  filepath = glob.glob(f"{output_path}/parse_result_fourth_fifth.csv")[0]#"/content/drive/MyDrive/security_mft/dfxml_files/parse_result_fourth_fifth.csv"
else:
  filepath = glob.glob(f"{output_path}/*.csv")[0]#"/content/drive/MyDrive/security_mft/dfxml_files/parse_result_fourth_fifth.csv"
print(filepath)
df = pd.read_csv(filepath)
print(f"How many columns: {len(df.columns)}")
print(f"column names: {df.columns}")
print(df.head(50))

/content/drive/MyDrive/security_mft/dfxml_files/parse_result_fourth_fifth.csv
How many columns: 64
column names: Index(['fileobject_id', 'alloc', 'atime', 'byte_run', 'byte_runs', 'crtime',
       'ctime', 'file_offset', 'filename', 'fileobject', 'filesize',
       'fs_offset', 'gid', 'hashdigest', 'hashdigest_fileobject', 'id',
       'img_offset', 'inode', 'inode_fileobject', 'len', 'meta_type', 'mode',
       'mtime', 'name_type', 'new_file', 'nlink', 'parent_object', 'partition',
       'seq', 'type', 'type_fileobject', 'uid', 'used',
       'alloc_original_fileobject', 'atime_original_fileobject',
       'changed_file', 'changed_property', 'changed_property_fileobject',
       'crtime_original_fileobject', 'ctime_original_fileobject',
       'deleted_file', 'file_offset_byte_runs', 'filename_original_fileobject',
       'filesize_original_fileobject', 'fs_offset_byte_runs',
       'gid_original_fileobject', 'hashdigest_original_fileobject',
       'id_original_fileobject', 'img_of

#analyzing

In [13]:
import matplotlib.pyplot as plt
import seaborn as sns

In [23]:
output_path = "/content/drive/MyDrive/security_mft/output/903202100/"
seed = "903202100"
# output_analysis = ""
#test only
# test_path = "/content/drive/MyDrive/security_mft/output/903202100/parse_result_fourth_fifth_byte_runs_903202100.csv"
# df = pd.read_csv(eachFile)

for eachFile in glob.glob(f"{output_path}/*{seed}.csv"):
  print(f"analyzing {eachFile} ...")
  df = pd.read_csv(eachFile)
  for column in df.columns:
    df_count_values = df[column].value_counts()
    print(df_count_values)
    csv_filename = f"{output_path}/parse_result_{os.path.basename(eachFile)[:-6]}_analysis.csv"
    df_count_values.to_csv (csv_filename, index = False, header=True)
  








analyzing /content/drive/MyDrive/security_mft/output/903202100/parse_result_fourth_fifth_903202100.csv ...
269    1
84     1
98     1
97     1
96     1
      ..
174    1
173    1
172    1
171    1
0      1
Name: fileobject_id, Length: 270, dtype: int64
1    240
0     30
Name: alloc, dtype: int64
2021-06-06T20:43:08Z    25
2021-06-06T20:43:18Z    18
2021-06-06T20:43:10Z    18
2021-06-06T20:43:11Z    16
2021-06-06T20:43:13Z    16
                        ..
2021-06-06T18:58:37Z     1
2021-06-06T20:08:46Z     1
2021-06-06T20:44:07Z     1
2021-06-06T18:58:34Z     1
2021-06-06T20:43:16Z     1
Name: atime, Length: 65, dtype: int64
Series([], Name: byte_runs, dtype: int64)
2021-06-06T20:11:52Z    11
2020-04-10T17:12:02Z    10
2019-03-19T04:52:44Z     9
2019-03-19T04:43:47Z     8
2020-04-10T17:12:23Z     6
                        ..
2019-03-19T06:25:36Z     1
2021-06-06T17:50:08Z     1
2021-06-06T18:58:21Z     1
2019-03-19T06:27:14Z     1
2021-06-06T20:28:04Z     1
Name: crtime, Length: 164, dt

In [17]:
df.value_counts()

Series([], dtype: int64)